In [4]:
import sys
from pathlib import Path

# Go from notebooks/ up to project root
project_root = Path("..").resolve()

# Point at the src/ folder
src_path = project_root / "src"

# Add src/ to sys.path so we can do `from dcf_model...`
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

print("Project root:", project_root)
print("Src path:", src_path)


Project root: /Users/norajohannessen/Git_ptojects/dcf/dcf-valuation
Src path: /Users/norajohannessen/Git_ptojects/dcf/dcf-valuation/src


In [5]:
import dcf_model.auto_metrics as am
import inspect

print("Module file:", am.__file__)
print("Public names:", [n for n in dir(am) if not n.startswith("_")])
print("Has compute_historical_metrics?", hasattr(am, "compute_historical_metrics"))


Module file: /Users/norajohannessen/Git_ptojects/dcf/dcf-valuation/src/dcf_model/auto_metrics.py
Public names: ['HistoricalMetrics', 'compute_cagr', 'compute_historical_metrics', 'dataclass', 'np', 'pd']
Has compute_historical_metrics? True


In [ ]:
import pandas as pd

from dcf_model.data_fetcher import build_company_financials_from_yahoo
from dcf_model.auto_metrics import compute_historical_metrics
from dcf_model.heuristics import build_assumption_suggestions
from dcf_model.assumptions import choose_scenario



# 1) Fetch and build financials from Yahoo
df_hist = build_company_financials_from_yahoo("AAPL")  # or "EQNR.OL", etc.

display(df_hist)

# 2) Compute metrics from history
metrics = compute_historical_metrics(df_hist)

# 3) Build bear/base/bull assumption suggestions
suggestions = build_assumption_suggestions(
    metrics,
    years=5,
    country="norway",   # adjust if needed
    sector="industrial" # or "tech", etc.
)

# 4) Choose a scenario
base_case = choose_scenario(
    {"bear": suggestions.bear, "base": suggestions.base, "bull": suggestions.bull},
    scenario="base",
)

print("Base-case WACC:", base_case.wacc)
print("Base-case terminal growth:", base_case.terminal_growth)
print("Base-case year 1 revenue growth:", base_case.growth.yearly_growth[1])


FileNotFoundError: [Errno 2] No such file or directory: 'data/processed/company_financials.csv'